In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import cv2
from utils.data_utils.acdc_datamodule import ACDCDataModule


In [ ]:
path = "database/training/patient001/patient001_frame01_gt.nii.gz"
p001_gt = nib.load(path).get_fdata()
p001_gt = np.transpose(p001_gt, (2,0,1) )

p001_gt.shape

In [ ]:
path = "database/training/patient001/patient001_frame01.nii.gz"
p001_img = nib.load(path).get_fdata()
p001_img = np.transpose(p001_img, (2,0,1) )

p001_img.shape

In [ ]:
plt.imshow(p001_img[1])

In [ ]:
plt.imshow(p001_gt[1])

In [ ]:
# split the multi-class masks into submasks
mask0 = np.where(p001_gt[:,:] == 0, 1,0)
mask1 = np.where(p001_gt[:,:] == 1, 1,0)
mask2 = np.where(p001_gt[:,:] == 2, 1,0)
mask3 = np.where(p001_gt[:,:] == 3, 1,0)
masked_img_0 = p001_img * mask0
masked_img_1 = p001_img * mask1
masked_img_2 = p001_img * mask2
masked_img_3 = p001_img * mask3
masked_images = [masked_img_0, masked_img_1, masked_img_2, masked_img_3]
masked_images[0].shape
masks = [mask0, mask1, mask2, mask3]


In [ ]:
plt.style.use('default')
fig, axes = plt.subplots(2,2, figsize=(12,12))
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(masked_images[i][1])
plt.show()

In [ ]:
def visualize(image_raw,mask):
    """
    iamge_raw:gray image with shape [width,height,1]
    mask: segment mask image with shape [num_class,width,height]
    this function return an image using multi color to visualize masks in raw image
    """
    # Convert grayscale image to RGB
    image = cv2.cvtColor(image_raw, cv2.COLOR_GRAY2RGB)
    
    # Get the number of classes (i.e. channels) in the mask
    num_class = mask.shape[0]
    
    # Define colors for each class (using a simple color map)
    colors = []
    for i in range(1, num_class):  # skip first class (background)
        hue = int(i/float(num_class-1) * 179)
        color = np.zeros((1, 1, 3), dtype=np.uint8)
        color[0, 0, 0] = hue
        color[0, 0, 1:] = 255
        color = cv2.cvtColor(color, cv2.COLOR_HSV2RGB)
        colors.append(color)

    # Overlay each non-background class mask with a different color on the original image
    for i in range(1, num_class):
        class_mask = mask[i, :, :]
        class_mask = np.repeat(class_mask[:, :, np.newaxis], 3, axis=2)
        class_mask = class_mask.astype(image.dtype)
        class_mask = class_mask * colors[i-1]
        image = cv2.addWeighted(image, 1.0, class_mask, 0.5, 0.0)

    return image

In [ ]:
masks = np.array(masks)
masks = np.transpose(masks, (1,0,2,3))
normalized_image = cv2.normalize(p001_img[1], None, 0, 255, cv2.NORM_MINMAX)
normalized_image = normalized_image.astype(np.uint8)
vis_img = visualize(normalized_image, masks[1])

plt.imshow(vis_img)

In [ ]:
plt.style.use('default')
fig, axes = plt.subplots(5,6, figsize=(16,12))
for i, ax in enumerate(axes.reshape(-1)):
    if i % 3 == 0:
        ax.imshow(p001_img[i//3,:,:])
    elif i % 3 == 1:
        ax.imshow(p001_gt[i//3,:,:])
    else:
        normalized_image = cv2.normalize(p001_img[i//3], None, 0, 255, cv2.NORM_MINMAX)
        normalized_image = normalized_image.astype(np.uint8)
        vis_img = visualize(normalized_image, masks[i//3])
        ax.imshow(vis_img)
plt.show()

In [ ]:
np.count_nonzero((p001_gt[1,:,:] == 0))
size = p001_gt[0].shape
size[0]*size[1]

In [ ]:
from utils.data_utils.data_utils import get_images_with_info
acdc_data = get_images_with_info("database/training")
acdc_test = get_images_with_info("database/testing")

In [ ]:
print('Number of learning images: ' + str(len(acdc_data[0])))

In [ ]:
print('Number of testing images: ' + str(len(acdc_test[0])))

## Infos about the patients

For each patient there is a 4D data which contains 12-35 record and we can see the exact value of this number in the NbFrame value. The fourth dimension means the time, and for each record we have about 10 frames spaced vertically.

We have 2 labeled records from these, one when the heart is full of blood (ED) and one when its not (ES).
The value for the ED and ES infos show which record it is.

Also we have 5 different groups of patients based on there healt, and we have information about their weight and height.

In [ ]:
import pandas as pd
info_df = pd.DataFrame.from_dict(acdc_data[2])

types = ['int64', 'int64', 'string', 'float32', 'int64', 'float32']
column_types_dict = dict(zip(info_df.columns, types))

info_df = info_df.astype(column_types_dict)

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(16,12))
for i, ax in enumerate(axes.reshape(-1)):
    ax.title.set_text(info_df.columns[i])
    info_df[info_df.columns[i]].value_counts().sort_index().plot(kind='bar', ax=ax)
plt.show()

## Percentages of pixels in labeled images grouped by the patient groups


In [ ]:
from utils.data_utils.data_utils import get_label_percentages
percs = []
for l in acdc_data[1]:
    lp = get_label_percentages(l[0])
    lp.append(l[1])
    lp.append(l[2])
    percs.append(lp)


In [ ]:
columns = ['background','RV','myocardium','LV','Group','Phase']
percs_df = pd.DataFrame(percs, columns=columns)

types = ['float32', 'float32', 'float32', 'float32', 'string', 'string']
column_types_dict = dict(zip(percs_df.columns, types))
percs_df = percs_df.astype(column_types_dict)

value_columns = percs_df.columns[0:4]
percs_df['Group'].value_counts()

In [ ]:
groups = info_df['Group'].value_counts().index
fig, axes = plt.subplots(2,3, figsize=(16,8))
for i, ax in enumerate(axes.reshape(-1)):
    if i < 5:
        ax.title.set_text(groups[i])
        data = percs_df[percs_df['Group'] == groups[i]]
        data = data[value_columns].mean()
        labels = []
        for l, v in zip(data.index , data.values):
            labels.append(format(v, ".2%") + " " + l)
        ax.pie(data.values, radius=1)
        ax.legend(labels, bbox_to_anchor=(0.9, 1), prop={'family': 'DejaVu Sans Mono'})
    else:
        ax.set_visible(False)
plt.show()

In [ ]:
percs_df['Phase'].value_counts()

In [ ]:
phases = percs_df['Phase'].value_counts().index
fig, axes = plt.subplots(1,2, figsize=(16,8))
for i, ax in enumerate(axes.reshape(-1)):
    ax.title.set_text(phases[i])
    data = percs_df[percs_df['Phase'] == phases[i]]
    data = data[value_columns].mean()
    labels = []
    for l, v in zip(data.index , data.values):
        labels.append(format(v, ".2%") + " " + l)
    ax.pie(data.values, radius=1)
    ax.legend(labels, bbox_to_anchor=(0.9, 1), prop={'family': 'DejaVu Sans Mono'})
plt.show()

### Variance of percentages

In [ ]:
percs_df[value_columns].std()